# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397743024906                   -0.89    5.2   50.1ms
  2   -8.400250905218       -2.60       -1.73    1.0   19.2ms
  3   -8.400405543230       -3.81       -2.90    1.8   20.7ms
  4   -8.400428687677       -4.64       -2.94    3.0   24.9ms
  5   -8.400428794383       -6.97       -2.99    1.0   19.2ms
  6   -8.400429017352       -6.65       -4.76    1.0   40.4ms
  7   -8.400429024053       -8.17       -4.45    3.2   26.0ms
  8   -8.400429024402       -9.46       -5.34    1.0   19.5ms
  9   -8.400429024418      -10.78       -6.51    1.0   19.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397839263438                   -0.90           5.2    523ms
  2   -8.400385799439       -2.59       -1.78   0.80    2.0    508ms
  3   -8.400423712509       -4.42       -3.00   0.80    1.0    184ms
  4   -8.400428969794       -5.28       -3.42   0.80    2.5   19.9ms
  5   -8.400429020098       -7.30       -4.61   0.80    1.2   16.5ms
  6   -8.400429024385       -8.37       -5.73   0.80    2.5   20.4ms
  7   -8.400429024420      -10.46       -5.96   0.80    2.8   44.4ms
  8   -8.400429024420      -12.71       -6.99   0.80    1.0   16.5ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.109688426927                   -1.02    2.77s
  2   -1.329857394108        0.39       -0.70    110ms
  3   -3.679546447584        0.37       -0.44   82.4ms
  4   -4.372989903998       -0.16       -0.49   40.5ms
  5   -6.453223954227        0.32       -0.56   42.0ms
  6   -7.358249330358       -0.04       -0.83   64.8ms
  7   -7.542035869660       -0.74       -1.33   29.7ms
  8   -7.951535224181       -0.39       -1.67   29.6ms
  9   -8.004523672491       -1.28       -1.74   29.7ms
 10   -8.119538521473       -0.94       -1.73   29.9ms
 11   -8.156290869132       -1.43       -2.13   33.5ms
 12   -8.256747445647       -1.00       -1.97   29.6ms
 13   -8.304472774703       -1.32       -2.09   29.8ms
 14   -8.336987556961       -1.49       -2.21   32.5ms
 15   -8.359532369602       -1.65       -2.11   29.8ms
 16   -8.386154087146       -1.57       -2.19   29.7ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397858033197                   -0.90    5.0   39.6ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400428845973                   -1.78    24.6s
  2   -8.400429024420       -6.75       -4.02    2.69s
  3   -8.400429024420      -14.75       -7.81    105ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.4072553344552214e-6
|ρ_newton - ρ_scfv| = 1.3498042024941967e-7
|ρ_newton - ρ_dm|   = 1.0227806099774543e-6
